In [15]:
import numpy as np
import cv2
import json

# Load the two images
warped_img = cv2.imread(r'C:\Users\Arun\pytorch\datasets\bills\wildreceipt\image_files\Image_2\0\74eb6a846a45613ebbd2379bcb1516b7227bbb0b.jpeg')
unwrapped_img = cv2.imread('unwrapped_img.png')

# Load annotations
with open('annotation.json', 'r') as f:
    annotations = json.load(f)
    
    
# Add your functions here
def standardize_box_heights(annotations):
    # Calculate average height of all boxes
    heights = []
    for annotation in annotations['annotations']:
        box = annotation['box']
        # Assuming box format is [x1,y1,x2,y2,x3,y3,x4,y4]
        height = ((box[3] - box[1]) + (box[5] - box[7])) / 2
        heights.append(height)
    
    # Get median height (more robust than mean)
    median_height = np.median([h for h in heights if h > 0])
    
    # Add a small padding
    std_height = median_height * 1.2
    
    # Adjust each box
    for annotation in annotations['annotations']:
        box = annotation['box']
        center_y = (box[1] + box[5]) / 2  # Find center y-coordinate
        
        # Update y-coordinates to use standard height
        half_height = std_height / 2
        box[1] = float(center_y - half_height)  # top y1
        box[3] = float(center_y - half_height)  # top y2
        box[5] = float(center_y + half_height)  # bottom y3
        box[7] = float(center_y + half_height)  # bottom y4
        
        annotation['box'] = box

def fix_overlapping_boxes(annotations):
    # Sort annotations by y-coordinate (top to bottom)
    sorted_annotations = sorted(annotations['annotations'], 
                               key=lambda a: (a['box'][1] + a['box'][5])/2)
    
    for i in range(len(sorted_annotations) - 1):
        current_box = sorted_annotations[i]['box']
        next_box = sorted_annotations[i+1]['box']
        
        # Get current bottom and next top
        current_bottom = max(current_box[5], current_box[7])
        next_top = min(next_box[1], next_box[3])
        
        # If overlapping
        if current_bottom > next_top:
            # Find midpoint between centers
            current_center = (current_box[1] + current_box[5]) / 2
            next_center = (next_box[1] + next_box[5]) / 2
            midpoint = (current_center + next_center) / 2
            
            # Adjust bottoms of current box
            current_box[5] = float(midpoint)
            current_box[7] = float(midpoint)
            
            # Adjust tops of next box
            next_box[1] = float(midpoint)
            next_box[3] = float(midpoint)
            
            # Update boxes
            sorted_annotations[i]['box'] = current_box
            sorted_annotations[i+1]['box'] = next_box
    
    # Update the original annotations
    annotations['annotations'] = sorted_annotations
    
def adjust_boxes_by_line(annotations):
    # Extract all boxes
    boxes = [annotation['box'] for annotation in annotations['annotations']]
    
    # Calculate center y-coordinate for each box
    centers_y = [(box[1] + box[3] + box[5] + box[7]) / 4 for box in boxes]
    
    # Group boxes by line (boxes with similar y-coordinates)
    tolerance = 10  # pixels
    lines = []
    current_line = [0]
    
    # Sort boxes by y-center
    indices = sorted(range(len(centers_y)), key=lambda i: centers_y[i])
    
    for i in range(1, len(indices)):
        idx = indices[i]
        prev_idx = indices[i-1]
        
        if abs(centers_y[idx] - centers_y[prev_idx]) < tolerance:
            current_line.append(idx)
        else:
            lines.append(current_line)
            current_line = [idx]
    
    if current_line:
        lines.append(current_line)
    
    # Adjust each line to have consistent height
    for line_indices in lines:
        if not line_indices:
            continue
            
        # Find appropriate height for this line
        heights = []
        for idx in line_indices:
            box = boxes[idx]
            heights.append(max(abs(box[1] - box[7]), abs(box[3] - box[5])))
        
        line_height = min(heights) * 0.4  # Use smallest height with some padding
        
        # Adjust all boxes in this line
        for idx in line_indices:
            center_y = centers_y[idx]
            half_height = line_height / 2
            
            box = boxes[idx]
            box[1] = center_y - half_height  # y1
            box[3] = center_y - half_height  # y2
            box[5] = center_y + half_height  # y3
            box[7] = center_y + half_height  # y4
            
            # Update the annotation
            annotations['annotations'][idx]['box'] = [float(val) for val in box]

# Find the homography matrix between the two images
# Using feature matching
gray1 = cv2.cvtColor(warped_img, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(unwrapped_img, cv2.COLOR_BGR2GRAY)

# Use SIFT or ORB for feature detection
sift = cv2.SIFT_create()  # or use ORB: cv2.ORB_create()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)

# Feature matching
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Apply ratio test to get good matches
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

# Find homography
if len(good_matches) >= 4:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    # Transform each bounding box
    for annotation in annotations['annotations']:
        box = annotation['box']
        # Reshape to points format for transformation
        points = np.array([[box[i], box[i+1]] for i in range(0, len(box), 2)], dtype=np.float32)
        points = points.reshape(-1, 1, 2)
        
        # After transforming the points
        transformed_points = cv2.perspectiveTransform(points, H)

        # Get the min/max coordinates to create an axis-aligned bounding box
        x_coords = [p[0][0] for p in transformed_points]
        y_coords = [p[0][1] for p in transformed_points]
        min_x, max_x = min(x_coords), max(x_coords)
        min_y, max_y = min(y_coords), max(y_coords)

        # Create new axis-aligned box (x1, y1, x2, y2, x3, y3, x4, y4)
        new_box = [min_x, min_y, max_x, min_y, max_x, max_y, min_x, max_y]
        # Convert all values to native Python floats
        annotation['box'] = [float(val) for val in new_box]
    
     # *** Add this part to call your functions ***
    # Option 1: Standardize heights
    # standardize_box_heights(annotations)
    
    # Option 2: Fix overlaps (you can use either or both)
    adjust_boxes_by_line(annotations)
    
    # Save the updated annotations
    with open('updated_annotation.json', 'w') as f:
        json.dump(annotations, f, indent=4)
else:
    print("Not enough good matches found")